In [24]:
%cd ..\src
!python setup.py develop

c:\Users\mscer\dev\EchoVPR\src
running develop
running egg_info
writing echovpr.egg-info\PKG-INFO
writing dependency_links to echovpr.egg-info\dependency_links.txt
writing top-level names to echovpr.egg-info\top_level.txt
reading manifest file 'echovpr.egg-info\SOURCES.txt'
writing manifest file 'echovpr.egg-info\SOURCES.txt'
running build_ext
Creating c:\users\mscer\anaconda3\envs\patchnetvlad\lib\site-packages\echovpr.egg-link (link to .)
echovpr 1.0 is already the active version in easy-install.pth

Installed c:\users\mscer\dev\echovpr\src
Processing dependencies for echovpr==1.0
Finished processing dependencies for echovpr==1.0


C:\Users\mscer\anaconda3\envs\patchnetvlad\lib\site-packages\setuptools\command\easy_install.py:156: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
C:\Users\mscer\anaconda3\envs\patchnetvlad\lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


In [25]:
from configs.utils import get_config
from echovpr.datasets.utils import load_np_file
import numpy as np

import logging
log = logging.getLogger(__name__)

In [26]:
# config = get_config("configs\\train_mlp_nordland_full.ini", log=False)

# nordland_summer_dataset_info = load_np_file(config['dataset_nordland_summer_dataset_file_path'])
# nordland_winter_dataset_info = load_np_file(config['dataset_nordland_winter_dataset_file_path'])
# val_test_splits = load_np_file(config['dataset_nordland_winter_val_test_splits_indices_file_path'])

In [27]:
config = get_config("configs\\train_esn_oxford.ini", log)['main']
for c in config:
    print(f"{c}:{config[c]}")

dataset:oxford
dataset_oxford_day_dataset_file_path:./data/oxford_day_dataset_d-20.npz
dataset_oxford_night_dataset_file_path:./data/oxford_night_dataset_d-20.npz
dataset_oxford_night_val_test_splits_indices_file_path:./data/oxford_night_valtest_splits_2.npz
dataset_oxford_day_hidden_repr_file_path:./data/oxford_day_hidden_repr.npy
dataset_oxford_night_hidden_repr_file_path:./data/oxford_night_hidden_repr.npy
dataloader_threads:1
train_batchsize:30
train_max_epochs:80
train_lr:0.0008
train_lr_sparce_divide_by:1000
model_in_features:500
model_out_features:4599
model_esn_num_connections:10
model_reservoir_size:6000
model_esn_alpha:0.3
model_esn_gamma:0.008
model_esn_rho:0.99
model_sparce_enabled:True
model_sparce_quantile:0.30
early_stopping_enabled:False
early_stopping_patience:10
early_stopping_min_delta:0.0001


In [28]:
oxford_day_dataset_info = load_np_file(config['dataset_oxford_day_dataset_file_path'])
oxford_night_dataset_info = load_np_file(config['dataset_oxford_night_dataset_file_path'])
val_test_splits = load_np_file(config['dataset_oxford_night_val_test_splits_indices_file_path'])

In [29]:
def export(image_names, file_name):
    with open(file_name,"w") as file:
        for image_name in image_names:
            file.writelines(image_name + "\n")

In [30]:
val_indices = val_test_splits['val_indices']
test_indices = val_test_splits['test_indices']

In [31]:
print(oxford_day_dataset_info['utm'].shape)
print(oxford_night_dataset_info['utm'].shape)
print(oxford_night_dataset_info['utm'].shape)
print(oxford_night_dataset_info['utm'][val_indices].shape)
print(oxford_night_dataset_info['utm'][test_indices].shape)
print(oxford_night_dataset_info['image_names'].shape)
print(oxford_night_dataset_info['image_names'][val_indices].shape)
print(oxford_night_dataset_info['image_names'][test_indices].shape)

# print(nordland_summer_dataset_info['image_indices'].shape)
# print(nordland_winter_dataset_info['image_indices'].shape)
# print(nordland_winter_dataset_info['image_indices'][val_indices].shape)
# print(nordland_winter_dataset_info['image_indices'][test_indices].shape)
# print(nordland_winter_dataset_info['image_names'].shape)
# print(nordland_winter_dataset_info['image_names'][val_indices].shape)
# print(nordland_winter_dataset_info['image_names'][test_indices].shape)

(4599, 2)
(4550, 2)
(4550, 2)
(455, 2)
(4095, 2)
(4550,)
(455,)
(4095,)


In [32]:
export(oxford_day_dataset_info['image_names'], "dataset_imagenames\\oxford_imageNames_day.txt")
export(oxford_night_dataset_info['image_names'], "dataset_imagenames\\oxford_imageNames_night.txt")
export(oxford_night_dataset_info['image_names'][val_indices], "dataset_imagenames\\oxford_imageNames_query_split_2_val.txt")
export(oxford_night_dataset_info['image_names'][test_indices], "dataset_imagenames\\oxford_imageNames_query_split_2_test.txt")

# export(nordland_summer_dataset_info['image_names'], "dataset_imagenames\\nordland_imageNames_summer.txt")
# export(nordland_winter_dataset_info['image_names'], "dataset_imagenames\\nordland_imageNames_winter.txt")
# export(nordland_winter_dataset_info['image_names'][val_indices], "dataset_imagenames\\nordland_imageNames_query_val.txt")
# export(nordland_winter_dataset_info['image_names'][test_indices], "dataset_imagenames\\nordland_imageNames_query_test.txt")

In [33]:
def export_ground_truth_patchnetvlad(file_name, utmDb, utmQ, posDistThr):
    # utmDb is the Day dataset for example for Oxford
    # utmQ is the Night dataset for example for Oxford but wither just validation or test set    
    np.savez_compressed("dataset_gt_files\\" + file_name, utmQ=utmQ, utmDb=utmDb, posDistThr=np.int64(posDistThr))

In [34]:
# export for Oxford
# export_ground_truth_patchnetvlad(f"oxford_d-{int(oxford_day_dataset_info['ground_truth_threshold'])}_val.npz", oxford_day_dataset_info['utm'], oxford_night_dataset_info['utm'][val_indices], oxford_day_dataset_info['ground_truth_threshold'])
# export_ground_truth_patchnetvlad(f"oxford_d-{int(oxford_day_dataset_info['ground_truth_threshold'])}_test.npz", oxford_day_dataset_info['utm'], oxford_night_dataset_info['utm'][test_indices], oxford_day_dataset_info['ground_truth_threshold'])

# export for Oxford - Split 2
export_ground_truth_patchnetvlad(f"oxford_d-{int(oxford_day_dataset_info['ground_truth_threshold'])}_split_2_val.npz", oxford_day_dataset_info['utm'], oxford_night_dataset_info['utm'][val_indices], oxford_day_dataset_info['ground_truth_threshold'])
export_ground_truth_patchnetvlad(f"oxford_d-{int(oxford_day_dataset_info['ground_truth_threshold'])}_split_2_test.npz", oxford_day_dataset_info['utm'], oxford_night_dataset_info['utm'][test_indices], oxford_day_dataset_info['ground_truth_threshold'])

# export for Nordland
# utmDb = np.repeat(np.expand_dims(nordland_summer_dataset_info['image_indices'], 1), 2, axis=1)
# utmQ = np.repeat(np.expand_dims(nordland_winter_dataset_info['image_indices'], 1), 2, axis=1)
# export_ground_truth_patchnetvlad(f"nordland_val.npz", utmDb, utmQ[val_indices], nordland_summer_dataset_info['ground_truth_threshold'])
# export_ground_truth_patchnetvlad(f"nordland_test.npz", utmDb, utmQ[test_indices], nordland_summer_dataset_info['ground_truth_threshold'])

In [38]:
import wandb
run = wandb.init()
artifact = run.use_artifact('uos_ml/echovpr_nordland/esn_iwa409cc:v0', type='model')
artifact_dir = artifact.download()
artifact_dir

wandb: Downloading large artifact esn_iwa409cc:v0, 1101.55MB. 2 files... Done. 0:0:0


'.\\artifacts\\esn_iwa409cc-v0'

In [35]:
# np.load(f"oxford_d-{int(oxford_day_dataset_info['ground_truth_threshold'])}_val.npz")['posDistThr']